In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import warnings
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import config


# EXTRACT DATA

In [2]:
# Study data files paths
average_scores_path = "Input_Data/naep.csv"
poverty_income_path = "Input_Data/poverty_income.csv"
unemployment_crime_path = "Input_Data/unemployment_crime.csv"
state_table_path = "Input_Data/state_table.csv"
total_revenue_path = "Input_Data/total_revenue_data.csv"
spending_percent_path = "Input_Data/percentage_instructional_spending_revenue.csv"
county_data_path = "Input_Data/county_data.csv"

# Read the score data and the revenue results
average_score_data = pd.read_csv(average_scores_path)
poverty_income_data = pd.read_csv(poverty_income_path)
unemployment_crime_data = pd.read_csv(unemployment_crime_path)
state_data = pd.read_csv(state_table_path)
total_revenue_data = pd.read_csv(total_revenue_path)
spending_percent_data = pd.read_csv(spending_percent_path)
county_data = pd.read_csv(county_data_path)

# TRANSFORM DATA

In [3]:
total_revenue_df = total_revenue_data
total_revenue_df

,year,state,total_revenue,total_expenditure_per_student
0,2013,Alabama,7153244000,8755
1,2013,Alaska,2545076000,18175
2,2013,Arizona,8097916000,7208
3,2013,Arkansas,5050784000,9394
4,2013,California,66445919000,9220
...,...,...,...,...
148,2017,Virginia,16529157000,11886
149,2017,Washington,15646461000,11989
150,2017,West Virginia,3476338000,11554
151,2017,Wisconsin,11686068000,11968


In [4]:
spending_percent_df = spending_percent_data
spending_percent_df

,year,state,total_revenue,total_instructional_spending,percent_spending_of_revenue
0,2013,Alabama,7153244000,3747993000,52.40
1,2013,Alaska,2545076000,1324649000,52.05
2,2013,Arizona,8097916000,3824071000,47.22
3,2013,Arkansas,5050784000,2545564000,50.40
4,2013,California,66445919000,34194878000,51.46
...,...,...,...,...,...
148,2017,Virginia,16529157000,9313733000,56.35
149,2017,Washington,15646461000,7613934000,48.66
150,2017,West Virginia,3476338000,1812983000,52.15
151,2017,Wisconsin,11686068000,6043577000,51.72


In [5]:
poverty_income_data = poverty_income_data.rename(columns = {'poverty_percent' : 'poverty_percentage', 'median_income' : 'med_income'})

poverty_income_data

,year,state,poverty_percentage,med_income
0,2019,Alabama,15.6,51771
1,2019,Alaska,10.2,77203
2,2019,Arizona,13.5,62027
3,2019,Arkansas,16.0,49020
4,2019,California,11.8,80423
...,...,...,...,...
454,2003,Virginia,10.0,50028
455,2003,Washington,11.0,48185
456,2003,West Virginia,16.3,32967
457,2003,Wisconsin,8.9,46538


In [6]:
# change column names to lower case for use in sql database
state_data = state_data.rename(columns = {'STATE_ID' : 'state_id', 'STATE' : 'state'})

state_data

,state_id,state
0,1,Alabama
1,2,Alaska
2,3,Arizona
3,4,Arkansas
4,5,California
5,6,Colorado
6,7,Connecticut
7,8,Delaware
8,9,District of Columbia
9,10,Florida


In [7]:
average_score_data

,year,state,avg_score,subject,grade
0,2019,Alabama,268.703899,Mathematics,8
1,2019,Alaska,274.337801,Mathematics,8
2,2019,Arizona,279.932022,Mathematics,8
3,2019,Arkansas,274.363192,Mathematics,8
4,2019,California,275.608909,Mathematics,8
...,...,...,...,...,...
1831,2003,Virginia,223.340359,Reading,4
1832,2003,Washington,221.096695,Reading,4
1833,2003,West Virginia,219.179525,Reading,4
1834,2003,Wisconsin,220.833428,Reading,4


In [8]:
# change column name to match poverty_income df
county_data = county_data.rename(columns = {'poverty_percent' : 'poverty_percentage'})
county_data

,year,state,county,poverty_percentage,med_income,unemployment_rate
0,2020,Alabama,Autauga County,11.2,67565,4.9
1,2020,Alabama,Baldwin County,8.9,71135,5.6
2,2020,Alabama,Barbour County,25.5,38866,7.0
3,2020,Alabama,Bibb County,17.8,50907,6.6
4,2020,Alabama,Blount County,13.1,55203,4.1
...,...,...,...,...,...,...
3136,2020,Wyoming,Sweetwater County,7.6,70583,7.4
3137,2020,Wyoming,Teton County,5.2,92488,6.0
3138,2020,Wyoming,Uinta County,8.5,71246,6.3
3139,2020,Wyoming,Washakie County,9.7,58532,5.3


In [9]:
# create year df
year_id =[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
year = [2003, 2005, 2007, 2009, 2011, 2013, 2015, 2017, 2019, 2020]
d = {'year_id': year_id, 'year': year}
year_df = pd.DataFrame(data=d)
year_df

,year_id,year
0,1,2003
1,2,2005
2,3,2007
3,4,2009
4,5,2011
5,6,2013
6,7,2015
7,8,2017
8,9,2019
9,10,2020


In [10]:
# create subject df
subject_id =[1, 2]
subject = ['Mathematics', 'Reading']
d = {'subject_id': subject_id, 'subject': subject}
subject_df = pd.DataFrame(data=d)
subject_df.head()

,subject_id,subject
0,1,Mathematics
1,2,Reading


In [11]:
# creat grade level df
grade_id =[1, 2]
grade = [4, 8]
d = {'grade_id': grade_id, 'grade': grade}
grade_df = pd.DataFrame(data=d)
grade_df.head()

,grade_id,grade
0,1,4
1,2,8


In [12]:
# merge score data with state, year, subject and grade data
merged_df = pd.merge(state_data, average_score_data, on='state')
merged_df = pd.merge(year_df, merged_df, on='year')
merged_df = pd.merge(subject_df, merged_df, on='subject')
merged_df = pd.merge(grade_df, merged_df, on='grade')
merged_df = merged_df.drop(columns = ['state', 'subject', 'year', 'grade'])
merged_df['avg_score'] = round(merged_df['avg_score'].apply(pd.to_numeric, errors = 'coerce'),2)

merged_df

,grade_id,subject_id,year_id,state_id,avg_score
0,1,1,1,1,223.34
1,1,1,1,2,232.99
2,1,1,1,3,228.91
3,1,1,1,4,229.01
4,1,1,1,5,227.45
...,...,...,...,...,...
1831,2,2,9,47,261.80
1832,2,2,9,48,266.28
1833,2,2,9,49,255.62
1834,2,2,9,50,267.38


In [13]:
# merge poverty and income data with state and year data
merged_income_df = pd.merge(state_data, poverty_income_data, on='state')
merged_income_df = pd.merge(year_df, merged_income_df, on='year')
merged_income_df = merged_income_df.drop(columns = ['state', 'year'])
merged_income_df['poverty_percentage'] = merged_income_df['poverty_percentage'].apply(pd.to_numeric, errors = 'coerce')
merged_income_df['med_income'] = merged_income_df['med_income'].astype(float)
merged_income_df

,year_id,state_id,poverty_percentage,med_income
0,1,1,15.3,36131.0
1,1,2,9.8,52391.0
2,1,3,13.9,41963.0
3,1,4,16.0,33445.0
4,1,5,13.7,48440.0
...,...,...,...,...
454,9,47,9.9,76471.0
455,9,48,9.8,78674.0
456,9,49,16.2,48659.0
457,9,50,10.4,64177.0


In [14]:
# merge revenue data with state and year data
merged_revenue_df = pd.merge(state_data, total_revenue_df, on='state')
merged_revenue_df = pd.merge(year_df, merged_revenue_df, on='year')
merged_revenue_df = merged_revenue_df.drop(columns = ['state', 'year'])
merged_revenue_df['total_revenue'] = merged_revenue_df['total_revenue'].apply(pd.to_numeric, errors = 'coerce')
merged_revenue_df['total_expenditure_per_student'] = merged_revenue_df['total_expenditure_per_student'].astype(float)
merged_revenue_df.head()

,year_id,state_id,total_revenue,total_expenditure_per_student
0,6,1,7153244000,8755.0
1,6,2,2545076000,18175.0
2,6,3,8097916000,7208.0
3,6,4,5050784000,9394.0
4,6,5,66445919000,9220.0


In [15]:
# merge spending data with state and year data
merged_spending_df = pd.merge(state_data, spending_percent_df, on='state')
merged_spending_df = pd.merge(year_df, merged_spending_df, on='year')
merged_spending_df = merged_spending_df.drop(columns = ['state', 'year'])
merged_spending_df['total_revenue'] = merged_spending_df['total_revenue'].apply(pd.to_numeric, errors = 'coerce')
merged_spending_df['total_instructional_spending'] = merged_spending_df['total_instructional_spending'].apply(pd.to_numeric, errors = 'coerce')
merged_spending_df['percent_spending_of_revenue'] = merged_spending_df['percent_spending_of_revenue'].apply(pd.to_numeric, errors = 'coerce')
merged_spending_df.head()

,year_id,state_id,total_revenue,total_instructional_spending,percent_spending_of_revenue
0,6,1,7153244000,3747993000,52.40
1,6,2,2545076000,1324649000,52.05
2,6,3,8097916000,3824071000,47.22
3,6,4,5050784000,2545564000,50.40
4,6,5,66445919000,34194878000,51.46


In [16]:
# merge unemployment and crime data with state and year data
merged_unemploy_crime_df = pd.merge(state_data, unemployment_crime_data, on='state')
merged_unemploy_crime_df = pd.merge(year_df, merged_unemploy_crime_df, on='year')
merged_unemploy_crime_df = merged_unemploy_crime_df.drop(columns = ['state', 'year'])
merged_unemploy_crime_df['unemployment_rate'] = merged_unemploy_crime_df['unemployment_rate'].apply(pd.to_numeric, errors = 'coerce')
merged_unemploy_crime_df['violent_crime_rate'] = merged_unemploy_crime_df['violent_crime_rate'].apply(pd.to_numeric, errors = 'coerce')
merged_unemploy_crime_df['property_crime_rate'] = merged_unemploy_crime_df['property_crime_rate'].apply(pd.to_numeric, errors = 'coerce')
merged_unemploy_crime_df

,year_id,state_id,unemployment_rate,violent_crime_rate,property_crime_rate
0,1,1,6.0,429.22,4046.45
1,1,2,7.8,598.04,3761.65
2,1,3,5.7,513.30,5634.03
3,1,4,5.9,456.38,3632.41
4,1,5,6.8,579.63,3426.38
...,...,...,...,...,...
454,9,47,2.7,209.38,1647.19
455,9,48,4.1,303.32,2725.75
456,9,49,4.9,318.89,1522.17
457,9,50,3.3,297.10,1473.00


In [17]:
# merge county data with state and year data
merged_county_df = pd.merge(state_data, county_data, on='state')
merged_county_df = pd.merge(year_df, merged_county_df, on='year')
merged_county_df = merged_county_df.drop(columns = ['state', 'year'])
merged_county_df['poverty_percentage'] = merged_county_df['poverty_percentage'].apply(pd.to_numeric, errors = 'coerce')
merged_county_df['med_income'] = merged_county_df['med_income'].apply(pd.to_numeric, errors = 'coerce')
merged_county_df['unemployment_rate'] = merged_county_df['unemployment_rate'].apply(pd.to_numeric, errors = 'coerce')
merged_county_df

,year_id,state_id,county,poverty_percentage,med_income,unemployment_rate
0,10,1,Autauga County,11.2,67565,4.9
1,10,1,Baldwin County,8.9,71135,5.6
2,10,1,Barbour County,25.5,38866,7.0
3,10,1,Bibb County,17.8,50907,6.6
4,10,1,Blount County,13.1,55203,4.1
...,...,...,...,...,...,...
3136,10,51,Sweetwater County,7.6,70583,7.4
3137,10,51,Teton County,5.2,92488,6.0
3138,10,51,Uinta County,8.5,71246,6.3
3139,10,51,Washakie County,9.7,58532,5.3


# LOAD DATA

In [18]:
# create database
dbURI = f"postgresql://{config.user}:{config.password}@{config.host}:{config.port}/{config.database}"
engine = create_engine(dbURI)

if not database_exists(engine.url):
    create_database(engine.url)
    
engine.execute('DROP SCHEMA IF EXISTS public CASCADE')
engine.execute('CREATE SCHEMA public')


In [19]:
with engine.begin() as connection:
    
    # create all tables from dataframes
    state_data.to_sql('state', con=connection, if_exists='replace', index=False)
    subject_df.to_sql('subject', con=connection, if_exists='replace',index=False)
    grade_df.to_sql('grade', con=connection, if_exists='replace',index=False)
    year_df.to_sql('year', con=connection, if_exists='replace',index=False)
    merged_income_df.to_sql('poverty_income', con=connection, if_exists='replace',index=True)
    merged_df.to_sql('average_score', con=connection, if_exists='replace',index=True)
    merged_revenue_df.to_sql('total_revenue', con=connection, if_exists='replace', index=True)
    merged_spending_df.to_sql('percent_spending', con=connection, if_exists='replace', index=True)
    merged_unemploy_crime_df.to_sql('unemployment_crime', con=connection, if_exists='replace', index=True)
    merged_county_df.to_sql('county_data', con=connection, if_exists='replace', index=True)
    
    # add primary keys to state, grade, year, subject tables
    connection.execute('ALTER TABLE state ADD PRIMARY KEY (state_id)')
    connection.execute('ALTER TABLE grade ADD PRIMARY KEY (grade_id)')
    connection.execute('ALTER TABLE year ADD PRIMARY KEY (year_id)')
    connection.execute('ALTER TABLE subject ADD PRIMARY KEY (subject_id)')
    
    # create foreign keys for poverty_income and avg_score table
    connection.execute('ALTER TABLE poverty_income ADD CONSTRAINT state_foreign FOREIGN KEY (state_id) REFERENCES state (state_id)')
    connection.execute('ALTER TABLE poverty_income ADD CONSTRAINT year_foreign FOREIGN KEY (year_id) REFERENCES year (year_id)')
    connection.execute('ALTER TABLE average_score ADD CONSTRAINT year_foreign FOREIGN KEY (year_id) REFERENCES year (year_id)')
    connection.execute('ALTER TABLE average_score ADD CONSTRAINT state_foreign FOREIGN KEY (state_id) REFERENCES state (state_id)')
    connection.execute('ALTER TABLE average_score ADD CONSTRAINT subject_foreign FOREIGN KEY (subject_id) REFERENCES subject (subject_id)')
    connection.execute('ALTER TABLE average_score ADD CONSTRAINT grade_foreign FOREIGN KEY (grade_id) REFERENCES grade (grade_id)')
    
    # rename indicies for poverty_income and avg_score tables
    connection.execute('ALTER TABLE poverty_income RENAME index TO id')
    connection.execute('ALTER TABLE average_score RENAME index TO id')
    connection.execute('ALTER TABLE poverty_income ADD PRIMARY KEY (id)')
    connection.execute('ALTER TABLE average_score ADD PRIMARY KEY (id)')
    
    # create primary and foreign keys for total_revenue table
    connection.execute('ALTER TABLE total_revenue RENAME index TO id')
    connection.execute('ALTER TABLE total_revenue ADD PRIMARY KEY (id)')
    connection.execute('ALTER TABLE total_revenue ADD CONSTRAINT state_foreign FOREIGN KEY (state_id) REFERENCES state (state_id)')
    connection.execute('ALTER TABLE total_revenue ADD CONSTRAINT year_foreign FOREIGN KEY (year_id) REFERENCES year (year_id)')
    
    # create primary and foreign keys for percent_spending table
    connection.execute('ALTER TABLE percent_spending RENAME index TO id')
    connection.execute('ALTER TABLE percent_spending ADD PRIMARY KEY (id)')
    connection.execute('ALTER TABLE percent_spending ADD CONSTRAINT state_foreign FOREIGN KEY (state_id) REFERENCES state (state_id)')
    connection.execute('ALTER TABLE percent_spending ADD CONSTRAINT year_foreign FOREIGN KEY (year_id) REFERENCES year (year_id)')
    
    # create primary and foreign keys for unemployment_crime table
    connection.execute('ALTER TABLE unemployment_crime ADD CONSTRAINT state_foreign FOREIGN KEY (state_id) REFERENCES state (state_id)')
    connection.execute('ALTER TABLE unemployment_crime ADD CONSTRAINT year_foreign FOREIGN KEY (year_id) REFERENCES year (year_id)')
    connection.execute('ALTER TABLE unemployment_crime RENAME index TO id')
    connection.execute('ALTER TABLE unemployment_crime ADD PRIMARY KEY (id)')
    
    # create primary and foreign keys for county_data table
    connection.execute('ALTER TABLE county_data ADD CONSTRAINT state_foreign FOREIGN KEY (state_id) REFERENCES state (state_id)')
    connection.execute('ALTER TABLE county_data ADD CONSTRAINT year_foreign FOREIGN KEY (year_id) REFERENCES year (year_id)')
    connection.execute('ALTER TABLE county_data RENAME index TO id')
    connection.execute('ALTER TABLE county_data ADD PRIMARY KEY (id)')